<a href="https://colab.research.google.com/github/Bhavesh0510/Logictrix_task/blob/main/Logictrix_AI_ML_task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import glob
total = 0
for i in glob.glob("/content/drive/MyDrive/Logictrix_task_2/train/*"):
    total = total + 1

print("Total images are:",total)

Total images are: 1671


In [7]:
from PIL import Image,ImageEnhance
import random
num = 0
for j in glob.glob("/content/drive/MyDrive/Logictrix_task_2/train/*"):
    img = Image.open(j)
    enhancer = ImageEnhance.Contrast(img)
    img_o = enhancer.enhance(random.uniform(0.1,0.5))
    img_o.save(f"/content/drive/MyDrive/Logictrix_task_2/train_dataset/low/{num}.jpg")
    num = num + 1


In [8]:
from PIL import Image,ImageEnhance
import random
num = 0
for j in glob.glob("/content/drive/MyDrive/Logictrix_task_2/train/*"):
    img = Image.open(j)
    enhancer = ImageEnhance.Contrast(img)
    img_o = enhancer.enhance(random.uniform(1.4,2.9))
    img_o.save(f"/content/drive/MyDrive/Logictrix_task_2/train_dataset/high/{num}.jpg")
    num = num + 1

In [13]:
import os
Main_DIR = "/content/drive/MyDrive/Logictrix_task_2/train_dataset/"
os.listdir(Main_DIR)

['high', 'low']

In [14]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [35]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from keras.models import Sequential
from keras.layers import Conv2D,ZeroPadding2D,BatchNormalization,Activation
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.preprocessing import image
from tensorflow.keras.preprocessing import image_dataset_from_directory


In [22]:
Batch_size = 32
Image_size = (320,320)
training_ds = image_dataset_from_directory(
    Main_DIR,
    batch_size = Batch_size,
    image_size = Image_size,
    shuffle = True,
    seed = 42,
    validation_split = 0.33,
    subset = "training",
)
validation_ds = image_dataset_from_directory(
    Main_DIR,
    batch_size = Batch_size,
    image_size = Image_size,
    shuffle = True,
    seed = 42,
    validation_split = 0.33,
    subset = "validation",
)

Found 3342 files belonging to 2 classes.
Using 2240 files for training.
Found 3342 files belonging to 2 classes.
Using 1102 files for validation.


In [26]:
AUTOTUNE = tf.data.AUTOTUNE
training_ds = training_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_ds = validation_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [27]:
normalize_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

In [28]:
training_ds = training_ds.map(lambda x,y: (normalize_layer(x),y))
validation_ds = validation_ds.map(lambda x,y: (normalize_layer(x), y))


In [37]:

def create_model(input_shape = (320,320,3)):
  x_input = Input(input_shape)
  x = ZeroPadding2D((3,3))(x_input)

  x = Conv2D(filters=16,kernel_size=(3,3),padding="same")(x)
  x = BatchNormalization(axis=3)(x)
  x = Activation("relu")(x)
  x = MaxPooling2D(pool_size=(2,2))(x)

  x = Conv2D(filters=32,kernel_size=(3,3),padding="same")(x)
  x = BatchNormalization(axis=3)(x)
  x = Activation("relu")(x)
  x = MaxPooling2D(pool_size=(2,2))(x)

  x = Conv2D(filters=64,kernel_size=(3,3),padding="same")(x)
  x = BatchNormalization(axis=3)(x)
  x = Activation("relu")(x)
  x = MaxPooling2D(pool_size=(2,2))(x)

  x = Conv2D(filters=128,kernel_size=(3,3),padding="same")(x)
  x = BatchNormalization(axis=3)(x)
  x = Activation("relu")(x)
  x = MaxPooling2D(pool_size=(2,2))(x)

  x = Flatten()(x)
  x = Dense(units=128, activation="relu")(x)
  x = Dense(units = 1)(x)

  mod = Model(inputs=x_input, outputs=x)
  return mod

In [38]:
model = create_model()

In [39]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics = ["accuracy"]
)

In [41]:
epochs = 5
history = model.fit(
    training_ds,
    epochs=epochs
)

Epoch 1/5
70/70 [==============================] - 342s 5s/step - loss: 0.2347 - accuracy: 0.9906
Epoch 2/5
70/70 [==============================] - 344s 5s/step - loss: 0.1202 - accuracy: 0.9933
Epoch 3/5
70/70 [==============================] - 350s 5s/step - loss: 0.0286 - accuracy: 0.9960
Epoch 4/5
70/70 [==============================] - 343s 5s/step - loss: 0.0472 - accuracy: 0.9973
Epoch 5/5
70/70 [==============================] - 345s 5s/step - loss: 0.1222 - accuracy: 0.9951
